# Sarath Kareti Invitae - Cohort Analysis Assignment

**In this analysis, I will display the code and perform EDA at each step and compile everything into one Class. The class will create a .csv. <br>
At the end of the analysis, we will be looking at a heatmap measuring retention rates across cohorts in each time period**

## Dependencies (Also in READ.me)

**Dependencies: Installation**

In [1]:
!pip3 install pandas
!pip3 install numpy
!pip3 install plotly
!pip3 install datetime

In [2]:
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
import datetime


In [3]:
pd.__version__

'1.2.4'

In [4]:
np.__version__

'1.20.1'

In [5]:
plotly.__version__

'5.7.0'

datetime is a built-in package in Python

## Step-By-Step

### Import Libraries

In [6]:
# import pandas as pd
# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
# import datetime
# from datetime import timedelta
# import math
# import timedelta
# import plotly.express as px
# %matplotlib inline

### Load Data

In [7]:
#change path according to users drive
orders = pd.read_csv("orders.csv")
customers = pd.read_csv("customers.csv")

In [8]:
#check if data loaded successfully 
print(orders.shape)
orders.head()

(27575, 4)


,id,order_number,user_id,created
0,1709,36,344,2014-10-28 00:20:01
1,1406,7,608,2014-10-14 23:44:53
2,1716,6,2296,2014-10-28 17:47:07
3,1426,2,1225,2014-10-15 18:33:38
4,1415,6,797,2014-10-15 02:07:16


In [9]:
#check if data loaded successfully 
print(customers.shape)
customers.head()

(25716, 2)


,id,created
0,35410,2015-07-03 22:01:11
1,35417,2015-07-03 22:11:23
2,35412,2015-07-03 22:02:52
3,35413,2015-07-03 22:05:02
4,35424,2015-07-03 22:21:55


In [10]:
#check for null values
orders.isnull().sum()

id              0
order_number    0
user_id         0
created         0
dtype: int64

In [11]:
customers.isnull().sum()

id         0
created    0
dtype: int64

In [12]:
#check for dtypes
customers.dtypes

id          int64
created    object
dtype: object

In [13]:
orders.dtypes

id               int64
order_number     int64
user_id          int64
created         object
dtype: object

## Preprocessing 

**We will ultimately not want id or user_id to be ints as we are not performing any calculations on them, so we will convert them to strings**

In [14]:
customers['id'] = customers['id'].astype(str).str.strip()
orders['id'] = orders['id'].astype(str).str.strip()
orders['user_id'] = orders['user_id'].astype(str).str.strip()

**Modify the 'created' columns in both tables to a pd.Datetime datatype as well as normalize them to remove timestamp in addition to creating %Y-%m-%d format**

In [15]:
orders

,id,order_number,user_id,created
0,1709,36,344,2014-10-28 00:20:01
1,1406,7,608,2014-10-14 23:44:53
2,1716,6,2296,2014-10-28 17:47:07
3,1426,2,1225,2014-10-15 18:33:38
4,1415,6,797,2014-10-15 02:07:16
...,...,...,...,...
27570,1408,17,497,2014-10-15 00:20:42
27571,1695,6,277,2014-10-27 02:42:24
27572,1715,1,1149,2014-10-28 17:24:36
27573,1418,32,344,2014-10-15 03:04:51


In [16]:
#datetime conversion
orders['Eastern'] = pd.to_datetime(orders['created'], errors = 'coerce').dt.tz_localize("US/Eastern", ambiguous = True, nonexistent = 'shift_forward')
customers['Eastern'] = pd.to_datetime(customers['created'], errors = 'coerce').dt.tz_localize("US/Eastern", ambiguous = True, nonexistent = 'shift_forward')


orders['created'] = pd.DatetimeIndex(orders.created).normalize()
customers['created'] = pd.DatetimeIndex(customers.created).normalize()
# # #assign to UTC 
# orders['UTC'] = orders['created'].dt.tz_localize('UTC')
# customers['UTC'] = customers['created'].dt.tz_localize('UTC')

# #convert to EST

# orders['Eastern'] = orders['created'].dt.tz_convert('US/Eastern')
# customers['Eastern'] = customers['created'].dt.tz_convert('US/Eastern')


# orders['created'] = orders['Eastern'].tz_localize(None)
# customers['created'] = customers['Eastern'].dt.tz_convert(None)


# customers.created

In [17]:
orders['created']

0       2014-10-28
1       2014-10-14
2       2014-10-28
3       2014-10-15
4       2014-10-15
           ...    
27570   2014-10-15
27571   2014-10-27
27572   2014-10-28
27573   2014-10-15
27574   2014-10-23
Name: created, Length: 27575, dtype: datetime64[ns]

In [18]:
customers.created

0       2015-07-03
1       2015-07-03
2       2015-07-03
3       2015-07-03
4       2015-07-03
           ...    
25711   2015-06-25
25712   2015-06-26
25713   2015-06-25
25714   2015-06-25
25715   2015-06-26
Name: created, Length: 25716, dtype: datetime64[ns]

***Check***

In [19]:
orders.created.dtypes

dtype('<M8[ns]')

In [20]:
orders.created.head(10)

0   2014-10-28
1   2014-10-14
2   2014-10-28
3   2014-10-15
4   2014-10-15
5   2014-10-27
6   2014-10-23
7   2014-10-26
8   2014-10-23
9   2014-10-28
Name: created, dtype: datetime64[ns]

In [21]:
customers.created.head(10)

0   2015-07-03
1   2015-07-03
2   2015-07-03
3   2015-07-03
4   2015-07-03
5   2015-07-04
6   2015-07-03
7   2015-07-03
8   2015-07-03
9   2015-07-03
Name: created, dtype: datetime64[ns]

**As of now, it appears that this preprocessing is sufficient, we will return when necessary**

## Join Tables

In [22]:
df = pd.merge(customers, orders, how = 'left', left_on = 'id', 
              right_on = 'user_id')
df.head()

,id_x,created_x,Eastern_x,id_y,order_number,user_id,created_y,Eastern_y
0,35410,2015-07-03,2015-07-03 22:01:11-04:00,NaN,NaN,NaN,NaT,NaT
1,35417,2015-07-03,2015-07-03 22:11:23-04:00,NaN,NaN,NaN,NaT,NaT
2,35412,2015-07-03,2015-07-03 22:02:52-04:00,NaN,NaN,NaN,NaT,NaT
3,35413,2015-07-03,2015-07-03 22:05:02-04:00,NaN,NaN,NaN,NaT,NaT
4,35424,2015-07-03,2015-07-03 22:21:55-04:00,27970,1.0,35424,2015-07-03,2015-07-03 23:37:49-04:00


In [23]:
#filter dataframe to keep necessary columns and also rename columns
df = df[['id_x', 'created_x', 'order_number', 'created_y']]

df.rename(columns = {"id_x": "user_id", "created_x":"account_creation", 
                     "created_y": "order_date"}, inplace = True)
print(df.shape)
df.head()



(37375, 4)


,user_id,account_creation,order_number,order_date
0,35410,2015-07-03,NaN,NaT
1,35417,2015-07-03,NaN,NaT
2,35412,2015-07-03,NaN,NaT
3,35413,2015-07-03,NaN,NaT
4,35424,2015-07-03,1.0,2015-07-03


In [24]:
#additionally, sort the dataframe by account_creation in DESC order, this will be important when we have to create cohorts
df = df.sort_values(by = 'account_creation', ascending = False)
df.head()



,user_id,account_creation,order_number,order_date
417,35798,2015-07-07,NaN,NaT
416,35815,2015-07-07,NaN,NaT
437,35836,2015-07-07,NaN,NaT
436,35829,2015-07-07,NaN,NaT
435,35806,2015-07-07,NaN,NaT


In [25]:
df.dtypes

user_id                     object
account_creation    datetime64[ns]
order_number               float64
order_date          datetime64[ns]
dtype: object

In [26]:
df.isnull().sum()

user_id                 0
account_creation        0
order_number        20360
order_date          20360
dtype: int64

**Since we want to count distinct users for each cohort, let's see if there are any duplicates prior to applying any functions.**

In [27]:
#mark first occurence of a useraa
df['new_user'] = df.user_id.duplicated().map({True:0, False:1})
df.new_user.value_counts()


1    25716
0    11659
Name: new_user, dtype: int64

***Verify by finding all rows in the dataframe where a user shows up onces vs. multiple times***

In [28]:
df.user_id.value_counts().tail(10)

28373    1
34839    1
21158    1
13178    1
10800    1
21150    1
34652    1
26567    1
17004    1
29674    1
Name: user_id, dtype: int64

In [29]:
df[df.user_id == '16481']

,user_id,account_creation,order_number,order_date,new_user
15141,16481,2015-02-27,NaN,NaT,1


In [30]:
df[df.user_id == '12205']

,user_id,account_creation,order_number,order_date,new_user
6784,12205,2015-01-16,15.0,2015-02-06,1
6785,12205,2015-01-16,19.0,2015-02-12,0
6786,12205,2015-01-16,20.0,2015-02-14,0
6787,12205,2015-01-16,21.0,2015-02-14,0
6788,12205,2015-01-16,4.0,2015-01-18,0
...,...,...,...,...,...
6774,12205,2015-01-16,18.0,2015-02-12,0
6777,12205,2015-01-16,3.0,2015-01-18,0
6778,12205,2015-01-16,2.0,2015-01-17,0
6779,12205,2015-01-16,8.0,2015-01-25,0


In [31]:
df['day_diff'] = df['order_date'].sub(df['account_creation'], fill_value = 0).dt.days

In [32]:
print("day_diff min:", df.day_diff.min())
print("day_diff max:", df.day_diff.max())

day_diff min: -16623
day_diff max: 186


**Looks like users who have never placed an order are creating negative values in this column, so when we create our bins for day_diff range we will have to account for this. We will create a temporary variable where if the value in day_diff is < 0, it will be assigned 0 in this new temporary column**

**Mark when someone created their first order**

In [33]:
df.order_number.value_counts()

1.0     5358
2.0     2796
3.0     1854
4.0     1345
5.0     1022
        ... 
90.0       1
77.0       1
63.0       1
94.0       1
65.0       1
Name: order_number, Length: 107, dtype: int64

In [34]:
df['first_order'] = np.where(df['order_number']==1.0, 1.0, 0.0)
df.first_order.value_counts()

0.0    32017
1.0     5358
Name: first_order, dtype: int64

**Since we want to capture all possible difference in intervals of n = 7, we need to make sure that our program captures the max value. In doing so we should check if the max value is divisible by 7 and if not, increase it so it is divisble until it is**

In [35]:
def closestNumber(n, m) :
    # Find the quotient
    q = int(n / m)
     
    # 1st possible closest number
    n1 = m * q
     
    # 2nd possible closest number
    if((n * m) > 0) :
        n2 = (m * (q + 1))
    else :
        n2 = (m * (q - 1))
     
    # if true, then n1 is the required closest number
    if (abs(n - n1) < abs(n - n2)) :
        return n1
     
    # else n2 is the required closest number
    return n2
#test
n = df.day_diff.max()
m = 7 #we want intervals of 7
new_max = closestNumber(n, m) #should return 189 as we want a number above our max to capture it in the  day range
new_max #189

189

**We will create bins with the range function and corresponding labels using string formatting**

In [36]:
bins = range(0, new_max+3, 7)
labels = [f'{a}-{b-1} Days' for a, b in zip(bins, bins[1:])]
#retain labels for later as we want to maintain order of day_diff_range for future tables
cols = labels

#apply pd cut using bins and new labels
df['day_diff_range'] = pd.cut(df['day_diff'], 
       bins=bins, 
       right=False, 
       labels=labels)
#correct for those 
#look at value counts
print(df.day_diff_range.value_counts().sum())
df.day_diff_range.value_counts()

17015


0-6 Days        3785
7-13 Days       1404
14-20 Days      1119
21-27 Days       975
28-34 Days       967
42-48 Days       815
35-41 Days       807
49-55 Days       715
56-62 Days       690
63-69 Days       659
70-76 Days       586
84-90 Days       536
77-83 Days       519
91-97 Days       474
98-104 Days      426
105-111 Days     418
112-118 Days     380
119-125 Days     333
126-132 Days     282
133-139 Days     254
140-146 Days     216
147-153 Days     201
154-160 Days     179
161-167 Days     149
168-174 Days      87
175-181 Days      37
182-188 Days       2
Name: day_diff_range, dtype: int64

In [37]:
df['day_diff_range'] = np.where(df['day_diff_range'].isnull(), 0, df.day_diff_range)
df.day_diff_range.value_counts()

0               20360
0-6 Days         3785
7-13 Days        1404
14-20 Days       1119
21-27 Days        975
28-34 Days        967
42-48 Days        815
35-41 Days        807
49-55 Days        715
56-62 Days        690
63-69 Days        659
70-76 Days        586
84-90 Days        536
77-83 Days        519
91-97 Days        474
98-104 Days       426
105-111 Days      418
112-118 Days      380
119-125 Days      333
126-132 Days      282
133-139 Days      254
140-146 Days      216
147-153 Days      201
154-160 Days      179
161-167 Days      149
168-174 Days       87
175-181 Days       37
182-188 Days        2
Name: day_diff_range, dtype: int64

**Now that we've created date ranges, we can create the cohorts**

In [38]:
#find the min and max dates of account creation. 1-pad the range so they are included in labels
date_min = df.account_creation.min() - datetime.timedelta(1)
date_max = df.account_creation.max() + datetime.timedelta(1)
bins = pd.date_range(date_min,date_max, freq = "7d")

labels = [f"{a}-{(b-datetime.timedelta(days = 1))}" for a, b in zip(bins, bins[1:])]

labels
type(labels[0])
df['Cohort'] = pd.cut(df['account_creation'], 
       bins=bins, 
       right=False,
       labels=labels)

#convert to strings
df['Cohort'] = [str(x) for x in df.Cohort]
# df = df[df.Cohort != 'nan']

# # # convert cohort to %Y-%m-%d using this function
# # #intution: since the cohort format is currently in yyyy-mm-dd-yyyy-mm-dd format and we want it in yyyy/mm/dd-yyyy/mm/dd format (without timestamps), we can create a dictionary of indices where the hyphens are (4,7,15,18) and replace them with a '/'. We want to keep the middle hyphen
def clean_cohort(string):
    
    x = string[0:10]
    y= string[20:30]
    z = x+'-'+y
    string = z
    cr = {4: '/',
          7: '/',
          15: '/',
         18:'/'}


    # Replace multiple characters with different replacement characters
    for index, replacement in cr.items():
        string = string[:index] + cr[index] + string[index + 1:]

    return string
df['Cohort'] = df.Cohort.map(lambda x: clean_cohort(x))

# # # # #look at value counts
print(df.Cohort.value_counts().sum())
df.Cohort.value_counts()

37375


2015/01/07-2015/01/13    3736
2015/02/25-2015/03/03    2827
2015/01/21-2015/01/27    2575
2015/03/04-2015/03/10    2180
2015/01/14-2015/01/20    1990
2015/03/11-2015/03/17    1656
2015/04/01-2015/04/07    1549
2015/04/22-2015/04/28    1487
2015/03/18-2015/03/24    1393
2015/03/25-2015/03/31    1349
2015/01/28-2015/02/03    1307
2015/05/20-2015/05/26    1138
2015/04/08-2015/04/14    1135
2015/07/01-2015/07/07    1125
2015/06/10-2015/06/16    1124
2015/06/03-2015/06/09    1078
2015/05/27-2015/06/02    1070
2015/06/17-2015/06/23    1045
2015/04/29-2015/05/05    1015
2015/04/15-2015/04/21    1004
2015/02/04-2015/02/10     926
2015/05/06-2015/05/12     906
2015/02/18-2015/02/24     889
2015/05/13-2015/05/19     850
2015/06/24-2015/06/30     826
2015/02/11-2015/02/17     773
2014/12/31-2015/01/06     422
Name: Cohort, dtype: int64

In [39]:
df.account_creation

417    2015-07-07
416    2015-07-07
437    2015-07-07
436    2015-07-07
435    2015-07-07
          ...    
1297   2015-01-01
1296   2015-01-01
1295   2015-01-01
1293   2015-01-01
1272   2015-01-01
Name: account_creation, Length: 37375, dtype: datetime64[ns]

In [40]:
df.day_diff.value_counts().sum()

37375

**Now that we have our Cohorts and date ranges, we can begin formatting and calculating metrics for Analysis**

In [41]:
df.Cohort

417     2015/07/01-2015/07/07
416     2015/07/01-2015/07/07
437     2015/07/01-2015/07/07
436     2015/07/01-2015/07/07
435     2015/07/01-2015/07/07
                ...          
1297    2014/12/31-2015/01/06
1296    2014/12/31-2015/01/06
1295    2014/12/31-2015/01/06
1293    2014/12/31-2015/01/06
1272    2014/12/31-2015/01/06
Name: Cohort, Length: 37375, dtype: object

**We want distinct users in our final dataset so we will drop duplicats of the combination of user_id and day_diff_range**

In [42]:
df.drop_duplicates(['user_id','day_diff_range'],inplace = True)

In [43]:
df.isnull().sum()

user_id                 0
account_creation        0
order_number        20360
order_date          20360
new_user                0
day_diff                0
first_order             0
day_diff_range          0
Cohort                  0
dtype: int64

**We want to look at users who have ordered items so we will mark those who have no order_number variable populated with a 0 and otherwise a 1**

In [44]:
df['ordered'] = np.where(df['order_number'].isnull(), 0, 1)
df.ordered.value_counts()

0    20360
1    14745
Name: ordered, dtype: int64

In [45]:
df2 = df[['new_user', 'Cohort', 'day_diff_range', 'first_order', 'ordered']]

df2.reset_index(inplace = True, drop = True)
print(df2.shape)
df2

(35105, 5)


,new_user,Cohort,day_diff_range,first_order,ordered
0,1,2015/07/01-2015/07/07,0,0.0,0
1,1,2015/07/01-2015/07/07,0,0.0,0
2,1,2015/07/01-2015/07/07,0,0.0,0
3,1,2015/07/01-2015/07/07,0,0.0,0
4,1,2015/07/01-2015/07/07,0,0.0,0
...,...,...,...,...,...
35100,1,2014/12/31-2015/01/06,0,0.0,0
35101,1,2014/12/31-2015/01/06,0,0.0,0
35102,1,2014/12/31-2015/01/06,0,0.0,0
35103,1,2014/12/31-2015/01/06,0,0.0,0


**We want to be able to see the total number of new users (new_user), how many users made their first order (first_order) and how many people ordered (ordered) within each Cohort and day_diff_range**

In [46]:
df2 = df2.groupby(['Cohort', 'day_diff_range']).sum().sort_values(by = 'Cohort', ascending = False).reset_index()
df2

,Cohort,day_diff_range,new_user,first_order,ordered
0,2015/07/01-2015/07/07,7-13 Days,0,0.0,1
1,2015/07/01-2015/07/07,0-6 Days,163,156.0,163
2,2015/07/01-2015/07/07,0,939,0.0,0
3,2015/06/24-2015/06/30,7-13 Days,18,12.0,31
4,2015/06/24-2015/06/30,14-20 Days,1,1.0,1
...,...,...,...,...,...
413,2014/12/31-2015/01/06,77-83 Days,1,2.0,7
414,2014/12/31-2015/01/06,84-90 Days,0,0.0,5
415,2014/12/31-2015/01/06,91-97 Days,1,1.0,5
416,2014/12/31-2015/01/06,98-104 Days,1,0.0,5


**For each cohort, create a cumulative sum fom the earliest to latest date**

In [47]:
df_new_user = df2.groupby(['Cohort'])['new_user'].sum().reset_index()
df_new_user.rename(columns = {'new_user':'Customers'}, inplace = True)
df_new_user.sort_values('Cohort', inplace = True, ascending = True)
df_new_user['Customers'] = df_new_user['Customers'].cumsum()
df_new_user

,Cohort,Customers
0,2014/12/31-2015/01/06,235
1,2015/01/07-2015/01/13,2297
2,2015/01/14-2015/01/20,3230
3,2015/01/21-2015/01/27,4550
4,2015/01/28-2015/02/03,5135
5,2015/02/04-2015/02/10,5609
6,2015/02/11-2015/02/17,6055
7,2015/02/18-2015/02/24,6483
8,2015/02/25-2015/03/03,8433
9,2015/03/04-2015/03/10,9890


**Merge these two dataframes together to acquire information for final table, we will configure in later steps**

In [48]:
df2 = pd.merge(df2, df_new_user, how = 'left', on = 'Cohort')
df2.sort_values(by = ['Cohort', 'day_diff_range'], ascending = False, inplace = True)

**Drop users who have not purchased anything**

In [49]:
df2 =df2[df2['day_diff_range']!=0]

**Assemble labels for the new customer percentage and order percentages to match the format on the instructions**

In [50]:
df2['new_perc'] = round(df2['new_user']/df2['Customers']*100, 3)
df2['order_perc'] = round(df2['ordered']/df2['Customers']*100, 3)

# #store above columns in tmp dataset
# df_tmp = df2[['new_perc', 'order_perc', 'new_user', 'ordered','new_perc', 'order_perc', 'Cohort']]

df2['new_customer_perc'] = round(df2['new_user']/df2['Customers']*100, 1).astype(str) + '% 1st time'
usr_labels = [f" ({str(x)})" for x in df2['new_user']]
df2['tmp_users'] = df2['new_customer_perc'] + usr_labels



df2['order_customer_perc'] = round(df2['ordered']/df2['Customers']*100, 1).astype(str) + '% orderers'
order_labels = [f" ({str(x)})" for x in df2['ordered']]
order_labels
df2['tmp_orders'] = df2['order_customer_perc'] + order_labels

**We want to store information in calculable forms for our visualization that shows retention rates**

In [51]:
#create retention dataframe for visualization
df_retention = df2[['Cohort', 'day_diff_range', 'ordered']]
df_retention

,Cohort,day_diff_range,ordered
0,2015/07/01-2015/07/07,7-13 Days,1
1,2015/07/01-2015/07/07,0-6 Days,163
3,2015/06/24-2015/06/30,7-13 Days,31
4,2015/06/24-2015/06/30,14-20 Days,1
5,2015/06/24-2015/06/30,0-6 Days,126
...,...,...,...
398,2014/12/31-2015/01/06,126-132 Days,9
399,2014/12/31-2015/01/06,119-125 Days,7
400,2014/12/31-2015/01/06,112-118 Days,3
401,2014/12/31-2015/01/06,105-111 Days,7


In [52]:
df2['combined']=df2['tmp_orders'].astype(str)+'\n'+df2['tmp_users']

In [53]:
df2['combined'] = np.where( (df2['new_perc'] == 0) & (df2['order_perc'] == 0 ), '', df2['combined'])
df2.combined

0          0.0% orderers (1)\n0.0% 1st time (0)
1      0.6% orderers (163)\n0.6% 1st time (163)
3        0.1% orderers (31)\n0.1% 1st time (18)
4          0.0% orderers (1)\n0.0% 1st time (1)
5      0.5% orderers (126)\n0.5% 1st time (120)
                         ...                   
398        3.8% orderers (9)\n0.9% 1st time (2)
399        3.0% orderers (7)\n0.0% 1st time (0)
400        1.3% orderers (3)\n0.0% 1st time (0)
401        3.0% orderers (7)\n0.4% 1st time (1)
403     11.9% orderers (28)\n7.7% 1st time (18)
Name: combined, Length: 391, dtype: object

In [54]:
df2 = df2[['Cohort', 'Customers', 'day_diff_range', 'combined']].sort_values('Cohort', ascending = False)
df3 = df2[['Cohort', 'Customers']]
df_pivoted = df2.pivot(index = 'Cohort', columns = 'day_diff_range', values = 'combined')
df_pivoted = df_pivoted.iloc[::-1]
df_pivoted.columns.name = None

df_pivoted = df_pivoted[cols]
df_pivoted.reset_index
df_final = pd.merge(df3, df_pivoted, how = 'left', on = 'Cohort')
df_final.drop_duplicates(['Cohort', 'Customers'], inplace = True)
df_final['Customers'] = df_final['Customers'].astype(str)+ ' Customers'
df_final

,Cohort,Customers,0-6 Days,7-13 Days,14-20 Days,21-27 Days,28-34 Days,35-41 Days,42-48 Days,49-55 Days,...,119-125 Days,126-132 Days,133-139 Days,140-146 Days,147-153 Days,154-160 Days,161-167 Days,168-174 Days,175-181 Days,182-188 Days
0,2015/07/01-2015/07/07,25716 Customers,0.6% orderers (163)\n0.6% 1st time (163),0.0% orderers (1)\n0.0% 1st time (0),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015/06/24-2015/06/30,24614 Customers,0.5% orderers (126)\n0.5% 1st time (120),0.1% orderers (31)\n0.1% 1st time (18),0.0% orderers (1)\n0.0% 1st time (1),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015/06/17-2015/06/23,23824 Customers,0.7% orderers (165)\n0.6% 1st time (152),0.2% orderers (45)\n0.1% 1st time (24),0.1% orderers (28)\n0.0% 1st time (11),0.0% orderers (1)\n0.0% 1st time (1),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2015/06/10-2015/06/16,22859 Customers,0.7% orderers (161)\n0.6% 1st time (128),0.2% orderers (52)\n0.1% 1st time (31),0.2% orderers (39)\n0.1% 1st time (22),0.1% orderers (25)\n0.1% 1st time (17),0.0% orderers (2)\n0.0% 1st time (1),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,2015/06/03-2015/06/09,21832 Customers,0.7% orderers (143)\n0.5% 1st time (114),0.2% orderers (46)\n0.1% 1st time (25),0.2% orderers (45)\n0.1% 1st time (21),0.2% orderers (34)\n0.1% 1st time (15),0.1% orderers (29)\n0.1% 1st time (15),0.0% orderers (1)\n0.0% 1st time (0),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,2015/05/27-2015/06/02,20902 Customers,0.6% orderers (127)\n0.5% 1st time (96),0.2% orderers (44)\n0.1% 1st time (19),0.2% orderers (40)\n0.1% 1st time (20),0.2% orderers (39)\n0.1% 1st time (18),0.2% orderers (38)\n0.1% 1st time (18),0.1% orderers (17)\n0.0% 1st time (9),0.0% orderers (1)\n0.0% 1st time (0),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,2015/05/20-2015/05/26,20000 Customers,0.6% orderers (129)\n0.4% 1st time (82),0.3% orderers (54)\n0.1% 1st time (22),0.2% orderers (49)\n0.1% 1st time (19),0.2% orderers (34)\n0.1% 1st time (14),0.2% orderers (44)\n0.1% 1st time (15),0.2% orderers (37)\n0.1% 1st time (15),0.1% orderers (25)\n0.1% 1st time (14),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,2015/05/13-2015/05/19,19103 Customers,0.5% orderers (98)\n0.3% 1st time (66),0.2% orderers (34)\n0.1% 1st time (19),0.2% orderers (30)\n0.0% 1st time (7),0.2% orderers (29)\n0.1% 1st time (11),0.2% orderers (30)\n0.1% 1st time (10),0.2% orderers (30)\n0.1% 1st time (15),0.1% orderers (27)\n0.1% 1st time (14),0.1% orderers (17)\n0.1% 1st time (11),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,2015/05/06-2015/05/12,18435 Customers,0.6% orderers (102)\n0.4% 1st time (74),0.2% orderers (30)\n0.1% 1st time (11),0.1% orderers (27)\n0.1% 1st time (11),0.2% orderers (30)\n0.1% 1st time (15),0.2% orderers (30)\n0.0% 1st time (7),0.2% orderers (28)\n0.0% 1st time (8),0.2% orderers (33)\n0.1% 1st time (12),0.1% orderers (22)\n0.0% 1st time (8),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,2015/04/29-2015/05/05,17738 Customers,0.7% orderers (119)\n0.4% 1st time (79),0.2% orderers (34)\n0.1% 1st time (12),0.2% orderers (35)\n0.0% 1st time (8),0.2% orderers (33)\n0.1% 1st time (15),0.2% orderers (33)\n0.0% 1st time (5),0.2% orderers (27)\n0.0% 1st time (8),0.2% orderers (33)\n0.1% 1st time (9),0.2% orderers (27)\n0.1% 1st time (9),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
df_final.set_index('Cohort', inplace = True)
df_final.fillna('', inplace = True)

In [56]:
df_final

,Customers,0-6 Days,7-13 Days,14-20 Days,21-27 Days,28-34 Days,35-41 Days,42-48 Days,49-55 Days,56-62 Days,...,119-125 Days,126-132 Days,133-139 Days,140-146 Days,147-153 Days,154-160 Days,161-167 Days,168-174 Days,175-181 Days,182-188 Days
Cohort,,,,,,,,,,,,,,,,,,,,,
2015/07/01-2015/07/07,25716 Customers,0.6% orderers (163)\n0.6% 1st time (163),0.0% orderers (1)\n0.0% 1st time (0),,,,,,,,...,,,,,,,,,,
2015/06/24-2015/06/30,24614 Customers,0.5% orderers (126)\n0.5% 1st time (120),0.1% orderers (31)\n0.1% 1st time (18),0.0% orderers (1)\n0.0% 1st time (1),,,,,,,...,,,,,,,,,,
2015/06/17-2015/06/23,23824 Customers,0.7% orderers (165)\n0.6% 1st time (152),0.2% orderers (45)\n0.1% 1st time (24),0.1% orderers (28)\n0.0% 1st time (11),0.0% orderers (1)\n0.0% 1st time (1),,,,,,...,,,,,,,,,,
2015/06/10-2015/06/16,22859 Customers,0.7% orderers (161)\n0.6% 1st time (128),0.2% orderers (52)\n0.1% 1st time (31),0.2% orderers (39)\n0.1% 1st time (22),0.1% orderers (25)\n0.1% 1st time (17),0.0% orderers (2)\n0.0% 1st time (1),,,,,...,,,,,,,,,,
2015/06/03-2015/06/09,21832 Customers,0.7% orderers (143)\n0.5% 1st time (114),0.2% orderers (46)\n0.1% 1st time (25),0.2% orderers (45)\n0.1% 1st time (21),0.2% orderers (34)\n0.1% 1st time (15),0.1% orderers (29)\n0.1% 1st time (15),0.0% orderers (1)\n0.0% 1st time (0),,,,...,,,,,,,,,,
2015/05/27-2015/06/02,20902 Customers,0.6% orderers (127)\n0.5% 1st time (96),0.2% orderers (44)\n0.1% 1st time (19),0.2% orderers (40)\n0.1% 1st time (20),0.2% orderers (39)\n0.1% 1st time (18),0.2% orderers (38)\n0.1% 1st time (18),0.1% orderers (17)\n0.0% 1st time (9),0.0% orderers (1)\n0.0% 1st time (0),,,...,,,,,,,,,,
2015/05/20-2015/05/26,20000 Customers,0.6% orderers (129)\n0.4% 1st time (82),0.3% orderers (54)\n0.1% 1st time (22),0.2% orderers (49)\n0.1% 1st time (19),0.2% orderers (34)\n0.1% 1st time (14),0.2% orderers (44)\n0.1% 1st time (15),0.2% orderers (37)\n0.1% 1st time (15),0.1% orderers (25)\n0.1% 1st time (14),,,...,,,,,,,,,,
2015/05/13-2015/05/19,19103 Customers,0.5% orderers (98)\n0.3% 1st time (66),0.2% orderers (34)\n0.1% 1st time (19),0.2% orderers (30)\n0.0% 1st time (7),0.2% orderers (29)\n0.1% 1st time (11),0.2% orderers (30)\n0.1% 1st time (10),0.2% orderers (30)\n0.1% 1st time (15),0.1% orderers (27)\n0.1% 1st time (14),0.1% orderers (17)\n0.1% 1st time (11),0.0% orderers (1)\n0.0% 1st time (1),...,,,,,,,,,,
2015/05/06-2015/05/12,18435 Customers,0.6% orderers (102)\n0.4% 1st time (74),0.2% orderers (30)\n0.1% 1st time (11),0.1% orderers (27)\n0.1% 1st time (11),0.2% orderers (30)\n0.1% 1st time (15),0.2% orderers (30)\n0.0% 1st time (7),0.2% orderers (28)\n0.0% 1st time (8),0.2% orderers (33)\n0.1% 1st time (12),0.1% orderers (22)\n0.0% 1st time (8),0.1% orderers (13)\n0.0% 1st time (9),...,,,,,,,,,,


**Format table**

In [57]:
s = df_final.style.set_table_styles([
                            {
                                "selector":"thead",
                                "props": [("background-color", "black"), ("color", "white"),
                                          ("border", "white"),
                                          ("font-size", "2rem"), ("font-style", "italic")]
                            },
                            {
                                "selector":"th.row_heading",
                                "props": [("background-color", "black"), ("color", "white"),
                                          ("border", "3px solid black"),
                                          ("font-size", "2rem"), ("font-style", "italic")]
                            },
                        ])

s

,Customers,0-6 Days,7-13 Days,14-20 Days,21-27 Days,28-34 Days,35-41 Days,42-48 Days,49-55 Days,56-62 Days,63-69 Days,70-76 Days,77-83 Days,84-90 Days,91-97 Days,98-104 Days,105-111 Days,112-118 Days,119-125 Days,126-132 Days,133-139 Days,140-146 Days,147-153 Days,154-160 Days,161-167 Days,168-174 Days,175-181 Days,182-188 Days
Cohort,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015/07/01-2015/07/07,25716 Customers,0.6% orderers (163) 0.6% 1st time (163),0.0% orderers (1) 0.0% 1st time (0),,,,,,,,,,,,,,,,,,,,,,,,,
2015/06/24-2015/06/30,24614 Customers,0.5% orderers (126) 0.5% 1st time (120),0.1% orderers (31) 0.1% 1st time (18),0.0% orderers (1) 0.0% 1st time (1),,,,,,,,,,,,,,,,,,,,,,,,
2015/06/17-2015/06/23,23824 Customers,0.7% orderers (165) 0.6% 1st time (152),0.2% orderers (45) 0.1% 1st time (24),0.1% orderers (28) 0.0% 1st time (11),0.0% orderers (1) 0.0% 1st time (1),,,,,,,,,,,,,,,,,,,,,,,
2015/06/10-2015/06/16,22859 Customers,0.7% orderers (161) 0.6% 1st time (128),0.2% orderers (52) 0.1% 1st time (31),0.2% orderers (39) 0.1% 1st time (22),0.1% orderers (25) 0.1% 1st time (17),0.0% orderers (2) 0.0% 1st time (1),,,,,,,,,,,,,,,,,,,,,,
2015/06/03-2015/06/09,21832 Customers,0.7% orderers (143) 0.5% 1st time (114),0.2% orderers (46) 0.1% 1st time (25),0.2% orderers (45) 0.1% 1st time (21),0.2% orderers (34) 0.1% 1st time (15),0.1% orderers (29) 0.1% 1st time (15),0.0% orderers (1) 0.0% 1st time (0),,,,,,,,,,,,,,,,,,,,,
2015/05/27-2015/06/02,20902 Customers,0.6% orderers (127) 0.5% 1st time (96),0.2% orderers (44) 0.1% 1st time (19),0.2% orderers (40) 0.1% 1st time (20),0.2% orderers (39) 0.1% 1st time (18),0.2% orderers (38) 0.1% 1st time (18),0.1% orderers (17) 0.0% 1st time (9),0.0% orderers (1) 0.0% 1st time (0),,,,,,,,,,,,,,,,,,,,
2015/05/20-2015/05/26,20000 Customers,0.6% orderers (129) 0.4% 1st time (82),0.3% orderers (54) 0.1% 1st time (22),0.2% orderers (49) 0.1% 1st time (19),0.2% orderers (34) 0.1% 1st time (14),0.2% orderers (44) 0.1% 1st time (15),0.2% orderers (37) 0.1% 1st time (15),0.1% orderers (25) 0.1% 1st time (14),,,,,,,,,,,,,,,,,,,,
2015/05/13-2015/05/19,19103 Customers,0.5% orderers (98) 0.3% 1st time (66),0.2% orderers (34) 0.1% 1st time (19),0.2% orderers (30) 0.0% 1st time (7),0.2% orderers (29) 0.1% 1st time (11),0.2% orderers (30) 0.1% 1st time (10),0.2% orderers (30) 0.1% 1st time (15),0.1% orderers (27) 0.1% 1st time (14),0.1% orderers (17) 0.1% 1st time (11),0.0% orderers (1) 0.0% 1st time (1),,,,,,,,,,,,,,,,,,
2015/05/06-2015/05/12,18435 Customers,0.6% orderers (102) 0.4% 1st time (74),0.2% orderers (30) 0.1% 1st time (11),0.1% orderers (27) 0.1% 1st time (11),0.2% orderers (30) 0.1% 1st time (15),0.2% orderers (30) 0.0% 1st time (7),0.2% orderers (28) 0.0% 1st time (8),0.2% orderers (33) 0.1% 1st time (12),0.1% orderers (22) 0.0% 1st time (8),0.1% orderers (13) 0.0% 1st time (9),,,,,,,,,,,,,,,,,,


## Create Class for Program

In [62]:
class Cohort_Analysis(object):

    
    def __init__(self,customer,order):
        self.customer:path = customer
        self.order:path = order
    
    
    def analysis(self):
        import pandas as pd
        import numpy as np
        import plotly.express as px
        import datetime
        
        
        def closestNumber(n:int, m:float):
            # Find the quotient
            q = int(n / m)

            # 1st possible closest number
            n1 = m * q

            # 2nd possible closest number
            if((n * m) > 0) :
                n2 = (m * (q + 1))
            else :
                n2 = (m * (q - 1))

            # if true, then n1 is the required closest number
            if (abs(n - n1) < abs(n - n2)) :
                return n1

            # else n2 is the required closest number
            return n2
        
        def clean_cohort(string:str):

            x = string[0:10]
            y= string[26:36]
            z = x+'-'+y
            string = z
            cr = {4: '/',
              7: '/',
              15: '/',
             18:'/'}
            # Replace multiple characters with different replacement characters
            for index, replacement in cr.items():
                string = string[:index] + cr[index] + string[index + 1:]
            return string
        
        print("Starting Analysis...")
        
        #read data in
        customers:pd.DataFrame = pd.read_csv(self.customer)
        orders:pd.DataFrame = pd.read_csv(self.order)
    
        #preprocess
        #datetime conversion
        orders['Eastern'] = pd.to_datetime(orders['created'], errors = 'coerce').dt.tz_localize("US/Eastern", ambiguous = True, nonexistent = 'shift_forward')
        customers['Eastern'] = pd.to_datetime(customers['created'], errors = 'coerce').dt.tz_localize("US/Eastern", ambiguous = True, nonexistent = 'shift_forward')


        orders['created'] = pd.DatetimeIndex(orders.created).normalize()
        customers['created'] = pd.DatetimeIndex(customers.created).normalize()
        #convert IDs to strings
        customers['id'] = customers['id'].astype(str).str.strip()
        orders['id'] = orders['id'].astype(str).str.strip()
        orders['user_id'] = orders['user_id'].astype(str).str.strip()
        
        #join tables
        df:pd.DataFrame = pd.merge(customers, orders, how = 'left', left_on = 'id', 
              right_on = 'user_id')
        #clean df
        df = df[['id_x', 'created_x', 'order_number', 'created_y']]
        df.rename(columns = {"id_x": "user_id", "created_x":"account_creation", 
        "created_y": "order_date"}, inplace = True)
        
        #store data in desc order based on account_creation
        df = df.sort_values(by = 'account_creation', ascending = False)
        
        #mark first occurrence of user to determine if they were a new customer in a given week or not
        df['new_user'] = df.user_id.duplicated().map({True:0, False:1})
        
        #calculate the difference in account creation and order placement date
        df['day_diff'] = df['order_date'].sub(df['account_creation'], fill_value = 0).dt.days
        
        
        #mark where each user placed their first order
        df['first_order'] = np.where(df['order_number']==1.0, 1.0, 0.0)
        
        #In this function, we calculate the max number that is divisible by 7 so each of our bins are of equally sized intervals
        n = df.day_diff.max()
        m = 7 #we want intervals of size 7

        new_max = closestNumber(n, m)
        
        #create ranges for the difference between account creation and order placement date
        bins = range(0, new_max+1, 7)
        labels = [f'{a}-{b-1} Days' for a, b in zip(bins, bins[1:])]
        cols = labels

        #apply pd cut using bins and new labels
        df['day_diff_range'] = pd.cut(df['day_diff'], 
               bins=bins, 
               right=False, 
               labels=labels)
        
        #account for NaT or NaN values that occur due to the above operation
        df['day_diff_range'] = np.where(df['day_diff_range'].isnull(), 0, df.day_diff_range)
        
        
        #now create Cohorts with a similar process as above
        date_min = df.account_creation.min() - datetime.timedelta(1)
        date_max = df.account_creation.max() + datetime.timedelta(1)
        bins = pd.date_range(date_min,date_max, freq = "7d")

        labels= [f"{a}-{(b-datetime.timedelta(days = 1))}" for a, b in zip(bins, bins[1:])]

        df['Cohort'] = pd.cut(df['account_creation'], 
               bins=bins, 
               right=False,
               labels=labels)

        #convert to strings
        df['Cohort'] = [str(x) for x in df.Cohort]
        
        df['Cohort'] = df.Cohort.map(lambda x: clean_cohort(x))
        
        df.drop_duplicates(['user_id','day_diff_range'],inplace = True)
        df['ordered'] = np.where(df['order_number'].isnull(), 0, 1)

        df2 = df[['new_user', 'Cohort', 'day_diff_range', 'first_order', 'ordered']]
        df2.reset_index(inplace = True, drop = True)
        df2 = df2.groupby(['Cohort', 'day_diff_range']).sum().sort_values(by = 'Cohort', ascending = False).reset_index()
        
        #dataframe to cumulate the total number of new users amonst each cohort
        df_new_user = df2.groupby(['Cohort'])['new_user'].sum().reset_index()
        df_new_user.rename(columns = {'new_user':'Customers'}, inplace = True)
        df_new_user.sort_values('Cohort', inplace = True, ascending = True)
        df_new_user['Customers'] = df_new_user['Customers'].cumsum()
        
        #merge dataframes together
        df2 = pd.merge(df2, df_new_user, how = 'left', on = 'Cohort')
        df2.sort_values(by = ['Cohort', 'day_diff_range'], ascending = False, inplace = True)
        #drop those who have not placed orders
        df2 =df2[df2['day_diff_range']!=0]
        
        #format dataframe 
        df2['new_perc'] = round(df2['new_user']/df2['Customers'], 3)
        df2['order_perc'] = round(df2['ordered']/df2['Customers'], 3)
        
        #before creating final dataset, create retention dataset for visualization
        df_retention:pd.DataFrame = df2[['Cohort', 'day_diff_range', 'ordered']]
        cohort_counts = df_retention.pivot(index = 'Cohort',
                                          columns = 'day_diff_range',
                                          values = 'ordered')
        cohort_sizes = cohort_counts.iloc[:, 0]
        retention = cohort_counts.divide(cohort_sizes, axis = 0)
        retention.columns.name = None
        retention = retention[cols]
        
        #create global to report later
        global fig
        retention = retention[::-1].round(3)*100
        fig = px.imshow(retention, aspect = 'auto',
               title = 'Cohort Retention Rates Across Time Intervals',
                       width=2000, height=1000, text_auto=True)
        fig.show()

        df2['new_customer_perc'] = round(df2['new_user']/df2['Customers']*100, 1).astype(str) + '% 1st time'
        usr_labels = [f" ({str(x)})" for x in df2['new_user']]
        df2['tmp_users'] = df2['new_customer_perc'] + usr_labels



        df2['order_customer_perc'] = round(df2['ordered']/df2['Customers']*100, 1).astype(str) + '% orderers'
        order_labels = [f" ({str(x)})" for x in df2['ordered']]
        order_labels
        df2['tmp_orders'] = df2['order_customer_perc'] + order_labels
        
        #combine information and separate with newline character
        df2['combined']=df2['tmp_orders'].astype(str)+'\n'+df2['tmp_users']
        
        #clean columns, put '' where nulls are
        df2['combined'] = np.where( (df2['new_perc'] == 0) & (df2['order_perc'] == 0 ), '', df2['combined'])
        
        
        df2 = df2[['Cohort', 'Customers', 'day_diff_range', 'combined']].sort_values('Cohort', ascending = False)
        df3 = df2[['Cohort', 'Customers']]
        df_pivoted = df2.pivot(index = 'Cohort', columns = 'day_diff_range', values = 'combined')
        df_pivoted = df_pivoted.iloc[::-1]
        df_pivoted.columns.name = None
        df_pivoted = df_pivoted[cols]
        df_pivoted.reset_index
        df_final = pd.merge(df3, df_pivoted, how = 'left', on = 'Cohort')
        df_final.drop_duplicates(['Cohort', 'Customers'], inplace = True)
        df_final['Customers'] = df_final['Customers'].astype(str)+ ' Customers'
        df_final.fillna('', inplace = True)
        df_final.set_index('Cohort', inplace = True)
        global df_final_styled
        df_final_styled = df_final.style.set_table_styles([
                            {
                                "selector":"thead",
                                "props": [("background-color", "black"), ("color", "white"),
                                          ("border", "white"),
                                          ("font-size", "2rem"), ("font-style", "italic")]
                            },
                            {
                                "selector":"th.row_heading",
                                "props": [("background-color", "black"), ("color", "white"),
                                          ("border", "3px solid black"),
                                          ("font-size", "2rem"), ("font-style", "italic")]
                            },
                        ])
        #export to csv
        df_final.to_csv("df_final.csv")
        return df_final_styled
        print('Finished!')

        
if __name__ == '__main__':
    import os
    customer = input("Please enter a valid file path to your customers file: ")
    while not os.path.isfile(customer):
        print("Error: That is not a valid file, try again...")
        customer = input("Please enter a valid file path to your customers file: ")

    order = input("Please enter a valid file path to your orders file: ")
    while not os.path.isfile(customer):
        print("Error: That is not a valid file, try again...")
        customer = input("Please enter a valid file path to your orders file: ")

    cohort = Cohort_Analysis(customer,order)
    cohort.analysis()

    
#call global variable df_final_styled to see final product. csv will be stored in the same path as the location of this file
df_final_styled


Please enter a valid file path to your customers file: customers.csv
Please enter a valid file path to your orders file: orders.csv
Starting Analysis...


,Customers,0-6 Days,7-13 Days,14-20 Days,21-27 Days,28-34 Days,35-41 Days,42-48 Days,49-55 Days,56-62 Days,63-69 Days,70-76 Days,77-83 Days,84-90 Days,91-97 Days,98-104 Days,105-111 Days,112-118 Days,119-125 Days,126-132 Days,133-139 Days,140-146 Days,147-153 Days,154-160 Days,161-167 Days,168-174 Days,175-181 Days,182-188 Days
Cohort,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015/07/01-7-07/00/00,25716 Customers,0.6% orderers (163) 0.6% 1st time (163),,,,,,,,,,,,,,,,,,,,,,,,,,
2015/06/24-6-30/00/00,24614 Customers,0.5% orderers (126) 0.5% 1st time (120),0.1% orderers (31) 0.1% 1st time (18),,,,,,,,,,,,,,,,,,,,,,,,,
2015/06/17-6-23/00/00,23824 Customers,0.7% orderers (165) 0.6% 1st time (152),0.2% orderers (45) 0.1% 1st time (24),0.1% orderers (28) 0.0% 1st time (11),,,,,,,,,,,,,,,,,,,,,,,,
2015/06/10-6-16/00/00,22859 Customers,0.7% orderers (161) 0.6% 1st time (128),0.2% orderers (52) 0.1% 1st time (31),0.2% orderers (39) 0.1% 1st time (22),0.1% orderers (25) 0.1% 1st time (17),,,,,,,,,,,,,,,,,,,,,,,
2015/06/03-6-09/00/00,21832 Customers,0.7% orderers (143) 0.5% 1st time (114),0.2% orderers (46) 0.1% 1st time (25),0.2% orderers (45) 0.1% 1st time (21),0.2% orderers (34) 0.1% 1st time (15),0.1% orderers (29) 0.1% 1st time (15),,,,,,,,,,,,,,,,,,,,,,
2015/05/27-6-02/00/00,20902 Customers,0.6% orderers (127) 0.5% 1st time (96),0.2% orderers (44) 0.1% 1st time (19),0.2% orderers (40) 0.1% 1st time (20),0.2% orderers (39) 0.1% 1st time (18),0.2% orderers (38) 0.1% 1st time (18),0.1% orderers (17) 0.0% 1st time (9),,,,,,,,,,,,,,,,,,,,,
2015/05/20-5-26/00/00,20000 Customers,0.6% orderers (129) 0.4% 1st time (82),0.3% orderers (54) 0.1% 1st time (22),0.2% orderers (49) 0.1% 1st time (19),0.2% orderers (34) 0.1% 1st time (14),0.2% orderers (44) 0.1% 1st time (15),0.2% orderers (37) 0.1% 1st time (15),0.1% orderers (25) 0.1% 1st time (14),,,,,,,,,,,,,,,,,,,,
2015/05/13-5-19/00/00,19103 Customers,0.5% orderers (98) 0.3% 1st time (66),0.2% orderers (34) 0.1% 1st time (19),0.2% orderers (30) 0.0% 1st time (7),0.2% orderers (29) 0.1% 1st time (11),0.2% orderers (30) 0.1% 1st time (10),0.2% orderers (30) 0.1% 1st time (15),0.1% orderers (27) 0.1% 1st time (14),0.1% orderers (17) 0.1% 1st time (11),,,,,,,,,,,,,,,,,,,
2015/05/06-5-12/00/00,18435 Customers,0.6% orderers (102) 0.4% 1st time (74),0.2% orderers (30) 0.1% 1st time (11),0.1% orderers (27) 0.1% 1st time (11),0.2% orderers (30) 0.1% 1st time (15),0.2% orderers (30) 0.0% 1st time (7),0.2% orderers (28) 0.0% 1st time (8),0.2% orderers (33) 0.1% 1st time (12),0.1% orderers (22) 0.0% 1st time (8),0.1% orderers (13) 0.0% 1st time (9),,,,,,,,,,,,,,,,,,


## Visualization: Retention Rates

***Retention Rates: The percentage of users who continue using a product or service over a given time period***

**Formula:**<br>
> [(E-N)/S]*100. = Retention Rate <br>
>Where,,,<br>
>>E= The number of total customers at the end of the time period<br>
>>N = The number of new customers added within the time period<br>
>>S = The number of existing customers at the start of the time period

In the heatmap below we are looking at retention rates in a given cohort across time. 

In [59]:
fig.show()

**Above is a heatmap produced with plotly.graph_objects. The heatamp is showing the retention rates amongst customers who have ordered a product in different cohorts across time. Based on the scale, brighter colors represent a higher retention rate whereas darker colors represent lower retention rates on a scale of 0.0%-100%. Across cohorts, it appears that retention is strong in the first few weeks of the Cohort's existence and then drops off. This may be because of the novelty effect where users keep retuning to the product or are referring their friends/family to the product as it is new and refreshing for them. Then, throughout time this novelty wears off and people stop returning to the company.**
<br>

**However, it does appear that this fall off is happening cyclically. Between the [0-6 days] and [7-13] days, it appears to be the greatest decrease in retention. But, moving from that the retention rate seems to be cycling up and down 30-40%. This means that the purchasing behavior of these users is healthy as in they are regularly coming back to purchase this product. The 2015/01/07 cohort is is interesting as it seems to have the highest retention rates. This is a prime example of cyclic buying amonst these users as we can see the retention rate drop from 41.4% in 63-69 days range to 27.7% in the 70-76 days range and leveling off to around 30% for most of the trajectory. Further investigation should inlude researching what specific products these users were purchasing and to see for what reaon (perhaps there is a drop down box asking why the user wanted to purchase the product**

<br>

***Possible suggestions to increase retention throughout time is to increase user engagement with the company by sending push notifications and asking for feedback. Within this feedback it is important to focus on what users did not like so that the company can focus on improving or even removing those features.***

## Closing Remarks

**Thank you so much for providing me with this opporutnity. I certainly learned a lot from this experience and enjoyed performing this analysis. I am extremely excited about potentially working at Invitae and joining the already talented community there! <br> <br>
Take Care,<br>
Sarath Kareti**